### The idea is to come up with an interactive dashboard where users of the dashboards can view players who are good at certain skills. For e.g. The list of players who are good at crossing, shooting etc.

### Import necessary libraries

In [11]:
import sqlite3
import pandas as pd

# Establish the connection to the db
cnx = sqlite3.connect('database.sqlite')

### Load the players data

In [12]:
# Loading the players table
players = pd.read_sql_query("SELECT * from Player", cnx)

In [13]:
# Show the first few rows and columns to have an idea of what the table looks like
players.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [14]:
# some basic validation
print(len(players['player_api_id']))
print(len(players['player_api_id'].unique()))

11060
11060


### Load the player attributes data

In [15]:
player_attributes = pd.read_sql_query("SELECT * from Player_Attributes", cnx)

In [16]:
# Having a glimpse of how the data looks like. It appears that the player attributes dataset was being
# updated regularly and so there is a latest date for each player attribute.
print(len(player_attributes))
player_attributes.head()

183978


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


### Obtain the most recent player attributes

In [17]:
player_attributes['date'] = pd.to_datetime(player_attributes['date'])
player_attribute_dates = player_attributes[['id', 'player_api_id', 'date']]

In [18]:
player_attribute_dates["rank"] = player_attribute_dates.groupby("player_api_id")["date"].rank(method="first", ascending=False)
player_attribute_dates = player_attribute_dates[player_attribute_dates['rank'] == 1.0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
# The length of player attributes was 183978 and has been reduced to 11060
# So let us confirm the transformation with an assetion on the length of unique player ids
# from player attributes and length of player_attribute_dates
# If the assertion is not True then we would get an AssertionError
assert len(player_attributes['player_api_id'].unique()) == len(player_attribute_dates['player_api_id'])

In [20]:
player_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


### Get the player info

In [21]:
players_attrs = player_attribute_dates.merge(player_attributes, on=['id', 'player_api_id', 'date'], how='left')
players_attrs.head()

,id,player_api_id,date,rank,player_fifa_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,505942,2016-02-18,1.0,218353,67.0,71.0,right,medium,medium,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,6,155782,2016-04-21,1.0,189615,74.0,76.0,left,high,medium,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
2,39,162549,2016-01-07,1.0,186170,65.0,67.0,right,medium,medium,...,68.0,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0
3,65,30572,2016-04-21,1.0,140161,69.0,69.0,right,medium,medium,...,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
4,88,23780,2015-12-24,1.0,17725,70.0,70.0,right,medium,medium,...,41.0,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0


In [22]:
player_info = pd.merge(players_attrs, players, on=['player_api_id', 'player_fifa_api_id'], how='left')
player_info.head()

,id_x,player_api_id,date,rank,player_fifa_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,...,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,id_y,player_name,birthday,height,weight
0,1,505942,2016-02-18,1.0,218353,67.0,71.0,right,medium,medium,...,6.0,11.0,10.0,8.0,8.0,1.0,Aaron Appindangoye,1992-02-29 00:00:00,182.88,187.0
1,6,155782,2016-04-21,1.0,189615,74.0,76.0,left,high,medium,...,14.0,7.0,9.0,9.0,12.0,2.0,Aaron Cresswell,1989-12-15 00:00:00,170.18,146.0
2,39,162549,2016-01-07,1.0,186170,65.0,67.0,right,medium,medium,...,16.0,11.0,12.0,9.0,13.0,3.0,Aaron Doran,1991-05-13 00:00:00,170.18,163.0
3,65,30572,2016-04-21,1.0,140161,69.0,69.0,right,medium,medium,...,15.0,12.0,13.0,12.0,11.0,4.0,Aaron Galindo,1982-05-08 00:00:00,182.88,198.0
4,88,23780,2015-12-24,1.0,17725,70.0,70.0,right,medium,medium,...,8.0,6.0,16.0,12.0,11.0,5.0,Aaron Hughes,1979-11-08 00:00:00,182.88,154.0


### Build an interactive dashboard

Install necessary libraries

In [23]:
!jupyter lab clean

[LabCleanApp] Cleaning /usr/local/share/jupyter/lab...
[LabCleanApp] Success!


In [24]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

  Using cached https://files.pythonhosted.org/packages/56/a0/dbcf5881bb2f51e8db678211907f16ea0a182b232c591a6d6f276985ca95/ipywidgets-7.5.1-py2.py3-none-any.whl
     |████████████████████████████████| 778kB 1.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/6c/7b/7ac231c20d2d33c445eaacf8a433f4e22c60677eb9776c7c5262d7ddee2d/widgetsnbextension-3.5.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/ab/872a23e29cec3cf2594af7e857f18b687ad21039c1f9b922fac5b9b142d5/traitlets-4.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/92/8fec943b5b81078399f969f00557804d884c96fcd0bc296e81a2ed4fd270/ipykernel-5.1.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/da/27/9a654d2b6cc1eaa517d1c5a4405166c7f6d72f04f6e7eea41855fe808a46/nbformat-4.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9a/41/220f49aaea88bc6fa6cba8d05ecf24676326156c23b991e80b3f2fc24c77/pickleshare-

  Using cached https://files.pythonhosted.org/packages/45/dc/3241eef99eb45f1def35cf93af35d1cf9ef4c0991792583b8f33ea41b092/more_itertools-7.2.0-py3-none-any.whl
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [25]:
%matplotlib notebook
import seaborn as sns
from ipywidgets import *
import numpy as np
import functools
import matplotlib.pyplot as plt

In [26]:
required_columns = ['player_name', 'height', 'weight']

In [27]:
required_numeric_columns = ['overall_rating',
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

In [28]:
def conjunction(*conditions):
    return functools.reduce(np.logical_or, conditions)

In [29]:
pd.options.mode.chained_assignment = None

In [35]:
def plot(**data):
    required_columns = ['player_name', 'height', 'weight']
    columns_to_display = required_columns + [column for column in data['columns_to_show']]
    del data['columns_to_show']
    columns = data.keys()
    comps = [player_info[column] > data[column] for column in columns]
    result = comps[0]
    for comp in comps[1:]:
        result &= comp
    df = player_info[result]
    re_order_numeric_columns = [item[0] for item in sorted(data.items(), key=lambda x:x[1], reverse=True)]
    df.sort_values(re_order_numeric_columns, ascending=False,inplace=True)
    display(df[columns_to_display])

sliders = {}

style = {'description_width': 'initial'}
for column in required_numeric_columns:
    sliders[column] = IntSlider(description=f'{column}', min=0, max=100, step=1, value=0, style=style)
sliders_per_row = 4
slider_displays = widgets.GridBox(list(sliders.values()), layout=widgets.Layout(grid_template_columns="repeat(4, 280px)"))

columns_to_show = widgets.SelectMultiple(
    options=required_numeric_columns,
    value=['crossing'],
    rows=10,
    description='Columns',
    disabled=False,
    layout=widgets.Layout(margin_left="0px")
)

sliders['columns_to_show'] = columns_to_show

dashboard_desc = """
This is an interactive dashboard to visualize the football dataset. 
The output is a simple table with the player name, height and weight.

The default value for all the sliders are set to 0.

The output table is sorted in descending order with the column/label that has the maximum value.
Let us say you have set crossing to 90 and finishing to 50, then the output table will be sorted with crossing first
and then finishing.You can use the columns multiple selection box to choose the columns you want to see.

"""


title = widgets.HTML(
    value="<H2 style=\"font-family:Verdana\"><center>Interactive visualization of the Football dataset</center></H2>",
)
description = widgets.HTML(
    value=f"<p style=\"font-family:Arial\">{dashboard_desc}</p><br>",
)
break_widget = widgets.HTML(
    value="<br>",
)

plot_output = widgets.interactive_output(plot, sliders)

title_widget = widgets.HBox([title])
description_widget = widgets.HBox([description])
table_widget = widgets.HBox([columns_to_show, plot_output])
dashboard = widgets.VBox([title_widget, description_widget, slider_displays, break_widget, table_widget])

In [34]:
display(dashboard)